In [19]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras import datasets as tfd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [4]:
def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)

In [27]:
train_data, test_data = tfd.cifar100.load_data(label_mode="coarse")
(x_train, y_train), (x_test, y_test) = train_data, test_data
x_train, x_test = np.mean(x_train, axis=3), np.mean(x_test, axis=3)

In [28]:
N_CLASSES = 20
INPUT_SIZE = (32, 32, 1)
LOSS = 'categorical_crossentropy'
METRICS = ['accuracy']
CALLBACKS = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True), tf.keras.callbacks.LearningRateScheduler(scheduler)]
BATCH_SIZE = 32
EPOCHS = 20

In [29]:
#Set shape to (num_samples, h, w, numchannels) = (40000, 32, 32, 1)
x_train = np.expand_dims(x_train, axis=-1) 
x_test = np.expand_dims(x_test, axis=-1)

# Convert labels to one-of-K (one-hot) encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=N_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=N_CLASSES)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

TRAIN_SIZE, _ , _ , _ = x_train.shape
input_img = tf.keras.Input(shape=INPUT_SIZE)
y_pred = None

In [ ]:
class_counts = np.sum(y_train, axis=0)
total_samples = len(y_train)
class_proportions = class_counts / total_samples

# Print the proportions for each class
for class_label, proportion in enumerate(class_proportions):
    print(f"Class {class_label}: Proportion - {proportion:.4f}")

In [25]:
import pickle
dict = pickle.load(open("cifar20_perturb_test.pkl", "rb"))
x_perturb, y_perturb = dict['x_perturb'], dict['y_perturb']
x_perturb = np.mean(x_perturb, axis=3)
x_perturb = np.expand_dims(x_perturb, axis=-1)
y_perturb = tf.keras.utils.to_categorical(y_perturb, num_classes=N_CLASSES)


In [24]:
def task1():
    fig, axs = plt.subplots(4, 5)
    axs = axs.flatten()
    for i, ax in enumerate(axs):
        print(x_train[i].shape)
        ax.imshow(x_train[i], cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()


def task_e_exploration():
    augmented_images = []
    num_images_to_augment = 20  # You can change this number based on your requirements

    # Applying augmentation to a subset of images
    for i in range(num_images_to_augment):
        augmented_img = data_augmenter.random_transform(x_train[i])
        print(augmented_img.shape)
        augmented_images.append(augmented_img)

    # Displaying the original and augmented images
    fig, axs = plt.subplots(4, 5)
    axs = axs.flatten()
    
    for i, ax in enumerate(axs):
        ax.imshow(augmented_images[i], cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()
    plt.show()


In [30]:
def E1():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh1)
    fh2 = tf.keras.layers.Dropout(0.1)(fh2)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh2)
    fh3 = tf.keras.layers.Dropout(0.1)(fh3)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)





#batch norm on all layers 
def E2():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.BatchNormalization()(h0)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.BatchNormalization()(h1)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.BatchNormalization()(h2)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh1)
    fh2 = tf.keras.layers.BatchNormalization()(fh2)
    fh2 = tf.keras.layers.Dropout(0.1)(fh2)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh2)
    fh3 = tf.keras.layers.BatchNormalization()(fh3)
    fh3 = tf.keras.layers.Dropout(0.1)(fh3)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)



#batch norm on dense layers only
def E3():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)

    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh1)
    fh2 = tf.keras.layers.BatchNormalization()(fh2)
    fh2 = tf.keras.layers.Dropout(0.1)(fh2)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh2)
    fh3 = tf.keras.layers.BatchNormalization()(fh3)
    fh3 = tf.keras.layers.Dropout(0.1)(fh3)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)

In [22]:
x_full, y_full = train_data
x_full = np.mean(x_full, axis=3)
x_full = np.expand_dims(x_full, axis=-1) 
y_full = tf.keras.utils.to_categorical(y_full, num_classes=N_CLASSES)

In [31]:
import os


# model.summary()

data_augmenter_spins = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                          width_shift_range=0.1, height_shift_range=0.1,
                                                          validation_split=0.2)

data_augmented_brightness = tf.keras.preprocessing.image.ImageDataGenerator(zca_whitening = True,  brightness_range= [0.5, 1], validation_split=0.2)

data_augmented_mixed = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                          width_shift_range=0.1, height_shift_range=0.1,
                                                          zca_whitening = True,  brightness_range= [0.5, 1],
                                                          validation_split=0.2)

sampler = tf.keras.preprocessing.image.ImageDataGenerator().flow(x_train, y_train, batch_size=BATCH_SIZE)
sampler_augmented_spins = data_augmenter_spins.flow(x_train, y_train, batch_size=BATCH_SIZE)
sampler_augmented_brightness = data_augmented_brightness.flow(x_train, y_train, batch_size=BATCH_SIZE)
sampler_augmented_mixed = data_augmented_mixed.flow(x_train, y_train, batch_size=BATCH_SIZE)

def train_and_evaluate_model(sampler, model_name, retrain=False, perturbation=False):
    model_save_path = f"{model_name}_model.h5"
    history_csv_path = f'results/{model_name}_training_history.csv'
    evaluation_csv_path = f'results/{model_name}_evaluation_results.csv'
    perturbation_csv_path = f'results/{model_name}_perturbed_results.csv'

    if not retrain and os.path.exists(model_save_path):
        # Load the existing model
        model = tf.keras.models.load_model(model_save_path)
        print(f"Loaded pre-trained model '{model_name}'.")

    else:
        model = tf.keras.Model(input_img, y_pred)
        print(model_name)

        optimizer = tf.keras.optimizers.Adam()

        model.compile(optimizer=optimizer, loss=LOSS, metrics=METRICS)
        history = model.fit(sampler, epochs=EPOCHS, steps_per_epoch=TRAIN_SIZE // BATCH_SIZE, callbacks=CALLBACKS, validation_data=(x_val, y_val))
        
        model.save(model_save_path)
        history_df = pd.DataFrame(history.history)
        history_df.to_csv(history_csv_path, index=False)

    test_loss, test_acc = model.evaluate(x_val, y_val)
  

    evaluation_df = pd.DataFrame({'Test Loss': [test_loss], 'Test Accuracy': [test_acc]})
    evaluation_df.to_csv(evaluation_csv_path, index=False)

    if perturbation:
        perturbation_loss, perturbation_acc = model.evaluate(x_perturb, y_perturb)
        perturbation_df = pd.DataFrame({'Perturbation Loss': [perturbation_loss], 'Perturbation Accuracy': [perturbation_acc]})
        perturbation_df.to_csv(perturbation_csv_path, index=False)

    return model



c:\Users\vdako\anaconda3\envs\sam-model-env\lib\site-packages\keras\src\preprocessing\image.py:1444: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn(


In [33]:
def C1():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh1)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu',  kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh2)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)

# Adding L2 reg with lambda=0.1 on the FC part
def C2():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(fh1)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu',  kernel_regularizer=tf.keras.regularizers.L2(0.1))(fh2)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)

# Adding L2 reg with lambda=0.001 in the FC part
def C3():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001))(fh1)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu',  kernel_regularizer=tf.keras.regularizers.L2(0.001))(fh2)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)

# Adding dropout with rate 0.5 in the FC part.
def C4():
    global y_pred
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu')(fh1)
    fh2 = tf.keras.layers.Dropout(0.5)(fh2)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu')(fh2)
    fh3 = tf.keras.layers.Dropout(0.5)(fh3)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)

# Adding dropout with rate 0.1 in the FC part.
def C5():
    global y_pred
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu')(fh1)
    fh2 = tf.keras.layers.Dropout(0.1)(fh2)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu')(fh2)
    fh3 = tf.keras.layers.Dropout(0.1)(fh3)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)


# Addinng dropout with 0.1 and L2 reg with lambda = 0.001 
def C6():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001))(fh1)
    fh2 = tf.keras.layers.Dropout(0.1)(fh2)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu',  kernel_regularizer=tf.keras.regularizers.L2(0.001))(fh2)
    fh3 = tf.keras.layers.Dropout(0.1)(fh3)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)

# Addinng dropout with 0.1 and L2 reg with lambda = 0.01
def C7():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh1)
    fh2 = tf.keras.layers.Dropout(0.1)(fh2)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(fh2)
    fh3 = tf.keras.layers.Dropout(0.1)(fh3)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)

# Adding dropout with 0.1 with L2 reg with lambda 0.1
def C8():
    global y_pred
    h0 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(input_img)
    h0 = tf.keras.layers.MaxPool2D((2,2))(h0)

    h1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(h0)
    h1 = tf.keras.layers.MaxPool2D((2,2))(h1)

    h2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(h1)
    h2 = tf.keras.layers.MaxPool2D((2,2))(h2)

    fh1 = tf.keras.layers.Flatten()(h2)
    fh2 = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(fh1)
    fh2 = tf.keras.layers.Dropout(0.1)(fh2)
    fh3 = tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(fh2)
    fh3 = tf.keras.layers.Dropout(0.1)(fh3)
    y_pred = tf.keras.layers.Dense(units=N_CLASSES, activation='softmax')(fh3)

In [34]:
C1()
model_name = 'C1'
train_and_evaluate_model(sampler, model_name, retrain=True, perturbation=False)
C2()
model_name = 'C2'
train_and_evaluate_model(sampler, model_name, retrain=True, perturbation=False)
C3()
model_name = 'C3'
train_and_evaluate_model(sampler, model_name, retrain=True, perturbation=False)
C4()
model_name = 'C4'
train_and_evaluate_model(sampler, model_name, retrain=True, perturbation=False)
C5()
model_name = 'C5'
train_and_evaluate_model(sampler, model_name, retrain=True, perturbation=False)
C6()
model_name = 'C6'
train_and_evaluate_model(sampler, model_name, retrain=True, perturbation=False)
C7()
model_name = 'C7'
train_and_evaluate_model(sampler, model_name, retrain=True, perturbation=False)
C8()
model_name = 'C8'
train_and_evaluate_model(sampler, model_name, retrain=True, perturbation=False)

C1
Epoch 1/20
1250/1250 [==============================] - 157s 122ms/step - loss: 4.2114 - accuracy: 0.2072 - val_loss: 2.9978 - val_accuracy: 0.2781 - lr: 0.0010
Epoch 2/20
1250/1250 [==============================] - 108s 87ms/step - loss: 2.6461 - accuracy: 0.3146 - val_loss: 2.4558 - val_accuracy: 0.3335 - lr: 0.0010
Epoch 3/20
1250/1250 [==============================] - 129s 103ms/step - loss: 2.2953 - accuracy: 0.3699 - val_loss: 2.2896 - val_accuracy: 0.3660 - lr: 0.0010
Epoch 4/20
1250/1250 [==============================] - 130s 104ms/step - loss: 2.1489 - accuracy: 0.4072 - val_loss: 2.1579 - val_accuracy: 0.4067 - lr: 0.0010
Epoch 5/20
1250/1250 [==============================] - 133s 107ms/step - loss: 2.0579 - accuracy: 0.4331 - val_loss: 2.1420 - val_accuracy: 0.4089 - lr: 0.0010
Epoch 6/20
1250/1250 [==============================] - 123s 99ms/step - loss: 1.9826 - accuracy: 0.4578 - val_loss: 2.1551 - val_accuracy: 0.4128 - lr: 0.0010
Epoch 7/20
1250/1250 [===========

c:\Users\vdako\anaconda3\envs\sam-model-env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 6s 18ms/step - loss: 2.0920 - accuracy: 0.4244
C2
Epoch 1/20
1250/1250 [==============================] - 75s 59ms/step - loss: 7.4376 - accuracy: 0.1624 - val_loss: 2.7648 - val_accuracy: 0.2387 - lr: 0.0010
Epoch 2/20
1250/1250 [==============================] - 73s 58ms/step - loss: 2.6489 - accuracy: 0.2545 - val_loss: 2.5109 - val_accuracy: 0.2849 - lr: 0.0010
Epoch 3/20
1250/1250 [==============================] - 70s 56ms/step - loss: 2.4833 - accuracy: 0.3002 - val_loss: 2.4615 - val_accuracy: 0.3169 - lr: 0.0010
Epoch 4/20
1250/1250 [==============================] - 92s 73ms/step - loss: 2.3761 - accuracy: 0.3345 - val_loss: 2.3466 - val_accuracy: 0.3395 - lr: 0.0010
Epoch 5/20
1250/1250 [==============================] - 125s 100ms/step - loss: 2.2814 - accuracy: 0.3595 - val_loss: 2.2727 - val_accuracy: 0.3661 - lr: 0.0010
Epoch 6/20
1250/1250 [==============================] - 113s 91ms/step - loss: 2.1993 - accuracy: 0.3839 - val

In [38]:
from tensorflow.keras.utils import plot_model
import visualkeras
M5()
model_name = 'M5'
model = tf.keras.Model(input_img, y_pred)
optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss=LOSS, metrics=METRICS)


visualkeras.layered_view(model).save(f'architecture_images/{model_name}.png')
# Plot the model architecture to a file (can be PNG, PDF, etc.)
